# Preprocessing of multiplexed scRNA-seq data via Cell Hashing

In this script, we will:
1. Demultiplex the data
2. Filter for the QC the individual models scRNA-seq data
3. Put the models' data together to obtain a unique Seurat object
4. SCTransform the merged object for normalization and variance stabilization
5. Calculate the cell cycle scoring for the merged object
6. Run the PCA followed by Harmony integration to resolve the batch effect
7. Run the UMAP for dimensionality reduction.

Loading the R packages avoiding messages.

In [1]:
suppressWarnings({suppressPackageStartupMessages({
  library(Seurat)
  library(ggplot2)
  library(readxl)
  library(AnnotationHub)
  library(ensembldb)
  library(tidyr)
  library(dplyr)
  library(viridis)
  library(harmony)
})})

Loading and demultiplexing the 10X Genomics count data for the three models: JHOS2 (representative cell line from primary sample), PDC1 (patient-derived cancer cells from a metastatic HGSOC sample), and PDC2 (patient-derived cancer cells from a primary HGSOC sample).

In [2]:
ch_demux <- function(path_to_10x, filename){
    # Loading data
    message("Loading data...")
    seurat_object_counts <- Read10X(data.dir = path_to_10x)
    seurat_object_HTO <- CreateSeuratObject(counts = seurat_object_counts$`Antibody Capture`)
    seurat_object_RNA <- CreateSeuratObject(counts = seurat_object_counts$`Gene Expression`, min.cells = 3, min.features = 200)

    message("Data loaded.")
    
    # Selecting cells for which both mRNA and HTO data is available
    message("Creating object...")
    joint.bcs <- intersect(colnames(seurat_object_HTO), colnames(seurat_object_RNA))
    
    message("Total number of cells for which both mRNA and HTO data is available: ", length(joint.bcs))
    
    seurat_object_RNA <- seurat_object_RNA[, joint.bcs]
    seurat_object_HTO <- seurat_object_HTO[, joint.bcs]

    # Dividing the cells into row and column HTO assays for practicality of the downstream assignment of the wells of origin
    seurat_object_HTO_row <- seurat_object_HTO[grep("row", rownames(seurat_object_HTO)), ]
    seurat_object_HTO_col <- seurat_object_HTO[grep("column", rownames(seurat_object_HTO)), ]
    seurat_object_HTO_demux <- CreateSeuratObject(counts = seurat_object_HTO_row[["RNA"]]@counts, assay = "HTO_row")
    seurat_object_HTO_demux[["HTO_col"]] <- CreateAssayObject(counts = seurat_object_HTO_col[["RNA"]]@counts) 
    
    message("Object created.")
    message("Starting data normalization.")
    
    # Normalizing the HTO data
    seurat_object_HTO_demux <- suppressMessages(NormalizeData(seurat_object_HTO_demux, 
                                                              assay = "HTO_row", normalization.method = "CLR"))
    seurat_object_HTO_demux <- suppressMessages(NormalizeData(seurat_object_HTO_demux, 
                                                              assay = "HTO_col", normalization.method = "CLR"))
    message("Data normalized")
    
    # Demultiplexing 
    message("Demultiplexing data...")
    
    seurat_object_HTO_demux <- suppressMessages(HTODemux(object = seurat_object_HTO_demux, 
                                                           assay = "HTO_row", 
                                                           positive.quantile = 0.99, 
                                                           kfunc = "kmeans"))
    seurat_object_HTO_demux <- suppressMessages(HTODemux(object = seurat_object_HTO_demux, 
                                                           assay = "HTO_col", 
                                                           positive.quantile = 0.99, 
                                                           kfunc = "kmeans"))
    
    message("Global classification results:")
    print(table(seurat_object_HTO_demux$HTO_row_classification.global))
    print(table(seurat_object_HTO_demux$HTO_col_classification.global))
    
    # Saving diagnostic plots before filtering
    message("Saving diagnostic plots before filtering for high-quality cells...")
    ggsave(HTOHeatmap(seurat_object_HTO_demux, assay = "HTO_row"), 
           filename = paste0(filename, "_row_HTO_before_filt_2023.pdf"), dpi = 1000, width = 5, height = 5) 
    ggsave(HTOHeatmap(seurat_object_HTO_demux, assay = "HTO_col"), 
           filename =  paste0(filename, "_col_HTO_before_filt_2023.pdf"), dpi = 1000, width = 5, height = 5)
    message("Plots saved.")
    
    # Classification
    classification <- rbind(seurat_object_HTO_demux$HTO_row_maxID, 
                            seurat_object_HTO_demux$HTO_col_maxID, 
                            seurat_object_HTO_demux$HTO_row_classification.global, 
                            seurat_object_HTO_demux$HTO_col_classification.global)
    rownames(classification) <- c("row", "column", "row_class", "col_class")
    classification <- as.data.frame(t(classification))
    
    # Saving the classification object for later
    assign(x = paste0(filename, "_classification"), value = classification, envir = .GlobalEnv)
    
    # Filtering for the high-quality cells, classified as "singlet"
    cells_retained <- rownames(classification[classification$row_class == "Singlet" & classification$col_class == "Singlet",])
    
    message("Number of cells retained: ", length(cells_retained))
    seurat_object_HTO_demux <- seurat_object_HTO_demux[, cells_retained]
    
    message("Data demultiplexed.")
    
    # Saving diagnostic plots after filtering
    message("Saving diagnostic plots after filtering for high-quality cells...")
    ggsave(HTOHeatmap(seurat_object_HTO_demux, assay = "HTO_row"), 
           filename = paste0(filename, "_row_HTO_after_filt_2023.pdf"), dpi = 1000, width = 5, height = 5) 
    ggsave(HTOHeatmap(seurat_object_HTO_demux, assay = "HTO_col"), 
           filename = paste0(filename, "_col_HTO_after_filt_2023.pdf"), dpi = 1000, width = 5, height = 5)
    message("Plots saved.")
    
    seurat_object <- seurat_object_RNA[, cells_retained]
    message("Process complete.")
    return(seurat_object)
}

In [3]:
JHOS2 <- ch_demux(path_to_10x = "./JHOS2_2023/outs/filtered_feature_bc_matrix/", filename = "JHOS2")
PDC1 <- ch_demux(path_to_10x = "./PDC1_2023/outs/filtered_feature_bc_matrix/", filename = "PDC1")
PDC2 <- ch_demux(path_to_10x = "./PDC2_2023/outs/filtered_feature_bc_matrix/", filename = "PDC2")

Loading data...

10X data contains more than one type and is being returned as a list containing matrices of each type.

Data loaded.

Creating object...

Total number of cells for which both mRNA and HTO data is available: 30634

Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from hto_col_ to htocol_"
Object created.

Starting data normalization.

Data normalized

Demultiplexing data...

Global classification results:




 Doublet Negative  Singlet 
    5200     9545    15889 

 Doublet Negative  Singlet 
    6398     9155    15081 


Saving diagnostic plots before filtering for high-quality cells...

Warning message:
"The `<scale>` argument of `guides()` cannot be `FALSE`. Use "none" instead as of ggplot2 3.3.4.
ℹ The deprecated feature was likely used in the Seurat package.
  Please report the issue at <https://github.com/satijalab/seurat/issues>."
Plots saved.

Number of cells retained: 13706

Data demultiplexed.

Saving diagnostic plots after filtering for high-quality cells...

Plots saved.

Process complete.

Loading data...

10X data contains more than one type and is being returned as a list containing matrices of each type.

Data loaded.

Creating object...

Total number of cells for which both mRNA and HTO data is available: 24251

Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from hto_col_ to htocol_"
Object created.

Starting data normalization.

Data normalized

Demultiplexing data...

Warning message:
"did not converge in 10 iterations"
Warn


 Doublet Negative  Singlet 
    5988     3109    15154 

 Doublet Negative  Singlet 
    7262     2458    14531 


Saving diagnostic plots before filtering for high-quality cells...

Plots saved.

Number of cells retained: 12144

Data demultiplexed.

Saving diagnostic plots after filtering for high-quality cells...

Plots saved.

Process complete.

Loading data...

10X data contains more than one type and is being returned as a list containing matrices of each type.

Data loaded.

Creating object...

Total number of cells for which both mRNA and HTO data is available: 27735

Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from hto_col_ to htocol_"
Object created.

Starting data normalization.

Data normalized

Demultiplexing data...

Warning message:
"did not converge in 10 iterations"
Global classification results:




 Doublet Negative  Singlet 
    5338     8704    13693 

 Doublet Negative  Singlet 
    6827     7781    13127 


Saving diagnostic plots before filtering for high-quality cells...

Plots saved.

Number of cells retained: 11501

Data demultiplexed.

Saving diagnostic plots after filtering for high-quality cells...

Plots saved.

Process complete.



Some QC metrics that we calculate are the % of counts assigned to the mitochondrial and ribosomal genes.

In [4]:
mt_rb_percentage <- function(seurat_object){
    seurat_object <- PercentageFeatureSet(seurat_object, pattern = "^MT-", col.name = "percent.mt")
    seurat_object <- PercentageFeatureSet(seurat_object, pattern = "^RP[SL]", col.name = "percent.rb")
    return(seurat_object)
}

In [5]:
JHOS2 <- mt_rb_percentage(JHOS2)
PDC1 <- mt_rb_percentage(PDC1)
PDC2 <- mt_rb_percentage(PDC2)

Plotting violin plots with counts' distribution.

In [6]:
plot_counts_vln <- function(seurat_object, filename){
    # Extrating the total counts for each cell
    n_count_before <- data.frame(n_counts = seurat_object$nCount_RNA)
    message("Saving histogram counts...")
    ggsave(ggplot(n_count_before, aes(x = n_counts)) + 
      geom_histogram(colour = "black", fill = "lightgrey", binwidth = 1000) + 
      geom_density(aes(y = ..count..*1000), alpha = .2, fill = "#FF6666") +
      ylab("") +
      ggtitle(filename) + 
      theme(panel.background = element_blank(),
            panel.grid = element_blank()),
      filename = paste0(filename, "_counts_histogram_2023.pdf"),
      width = 5,
      height = 5,
      dpi = 1000)
    message("Done. Saving QC violin plots...")
    
    ggsave(VlnPlot(seurat_object, 
           features = c("nFeature_RNA", "nCount_RNA", "percent.mt", "percent.rb"), ncol = 4, pt.size = 0.0001),
           filename = paste0(filename, "_QC_violin_plot_2023.pdf"),
           width = 8, height = 5, dpi = 1000) 
    message("Done.")
}

In [7]:
plot_counts_vln(JHOS2, "JHOS2")
plot_counts_vln(PDC1, "PDC1")
plot_counts_vln(PDC2, "PDC2")

Saving histogram counts...

Warning message:
"The dot-dot notation (`..count..`) was deprecated in ggplot2 3.4.0.
ℹ Please use `after_stat(count)` instead."
Done. Saving QC violin plots...

Done.

Saving histogram counts...

Done. Saving QC violin plots...

Done.

Saving histogram counts...

Done. Saving QC violin plots...

Done.



Performing QC filtering.

In [8]:
JHOS2 <- subset(JHOS2, subset = nCount_RNA > 2500 & nCount_RNA < 80000 & percent.mt < 25)
PDC1 <- subset(PDC1, subset = nCount_RNA > 2500 & nCount_RNA < 80000 & percent.mt < 20)
PDC2 <- subset(PDC2, subset = nCount_RNA > 2500 & nCount_RNA < 80000 & percent.mt < 30)

Annoating the sample names.

In [9]:
treatment_groups <- as.data.frame(read_xlsx(path = "/research/ungureanulab/AD_cellhashing_results/Treatment_groups.xlsx", 
                                      sheet = 1, col_names = T))
rownames(treatment_groups) <- treatment_groups$`HTO classification`

In [10]:
group_assignment <- function(seurat_object, ass_object, filename){
    message("Assignment of the groups...")
    classification <- get(x = paste0(filename, "_classification"))
    wells_classification <- sapply(rownames(seurat_object@meta.data), 
                                     function(x) paste0(classification[x, "column"], "_", classification[x, "row"]))
                              
    write.table(x = table(wells_classification), file = paste0(filename, "_n_cells_per_HTO_classification.txt"), 
                sep = "\t", 
                quote = F,
                row.names = F)
    write.table(x = wells_classification, file = paste0(filename, "_HTO_classification_cell_by_cell.txt"), 
                sep = "\t", 
                quote = F,
                row.names = T, col.names = F)

    if("Drug" %in% colnames(ass_object)){
        seurat_object@meta.data[["Treatment_group"]] <- sapply(wells_classification, function(x) treatment_groups[x, "Drug"])
        write.table(table(seurat_object@meta.data[["Treatment_group"]]), 
                    file = paste0(filename, "_cells_treatment_group.txt"), 
                    sep = "\t", quote = F, row.names = F)
        
        message("Done.")                                                       
        return(seurat_object)    
    }else{
        message("Reformat your dataframe.")
    }                                                           
}

In [11]:
JHOS2 <- group_assignment(seurat_object = JHOS2, ass_object = treatment_groups, filename = "JHOS2")
PDC1 <- group_assignment(seurat_object = PDC1, ass_object = treatment_groups, filename = "PDC1")
PDC2 <- group_assignment(seurat_object = PDC2, ass_object = treatment_groups, filename = "PDC2")

Assignment of the groups...

Done.

Assignment of the groups...

Done.

Assignment of the groups...

Done.



Now we will merge the objects, but before that, we need to assign individual cell names to the cells.

In [12]:
JHOS2 <- RenameCells(JHOS2, add.cell.id = "JHOS2") 
PDC1 <- RenameCells(PDC1, add.cell.id = "PDC1") 
PDC2 <- RenameCells(PDC2, add.cell.id = "PDC2") 
hgsoc_merged.sct <- merge(x = JHOS2, y = c(PDC1, PDC2))

Launching SCTransform.

In [13]:
hgsoc_merged.sct <- SCTransform(hgsoc_merged.sct,
                                vars.to.regress = c("percent.rb", # Calculated on the individual objects, no need to recalculate it on the merged object, since they are calculated in the same way and will be regressed out anyway
                                                    "percent.mt", 
                                                    "nFeature_RNA", 
                                                    "nCount_RNA"),  
                                method = "glmGamPoi",
                                return.only.var.genes = FALSE, 
                                variable.features.n = 2000,
                                vst.flavor = "v2", verbose = FALSE)

Calculating the cell cycle scoring, then re-normalizing. The cell cycle genes were retrieved from Tirosh *et al.* (2016) https://www.science.org/doi/10.1126/science.aad0501

In [14]:
cell_cycle_genes <- read.csv(file = "/research/ungureanulab/AD_cellhashing_results/Homo_sapiens_cell_cycle.csv")
ah <- AnnotationHub()
ahDb <- query(ah, 
              pattern = c("Homo sapiens", "EnsDb"), 
              ignore.case = TRUE)
id <- ahDb %>%
  mcols() %>%
  rownames() %>%
  tail(n = 1)
edb <- ah[[id]]
annotations <- genes(edb, return.type = "data.frame")
annotations <- annotations %>%
  dplyr::select(gene_id, gene_name, seq_name, gene_biotype, description)
cell_cycle_markers <- dplyr::left_join(cell_cycle_genes, annotations, by = c("geneID" = "gene_id"))
s_genes <- cell_cycle_markers %>%
  dplyr::filter(phase == "S") %>%
  pull("gene_name")
g2m_genes <- cell_cycle_markers %>%
  dplyr::filter(phase == "G2/M") %>%
  pull("gene_name")

hgsoc_merged.sct <- CellCycleScoring(
  hgsoc_merged.sct,
  s.features = s_genes,
  g2m.features = g2m_genes,
  assay = "SCT",
  set.ident = TRUE)

# Re-launching on the RNA assay, which is set as default by the function
hgsoc_merged.sct <- SCTransform(hgsoc_merged.sct,
                                vars.to.regress = c("percent.rb", 
                                                    "percent.mt", 
                                                    "nFeature_RNA", 
                                                    "nCount_RNA", 
                                                    "S.Score", 
                                                    "G2M.Score"),  
                                method = "glmGamPoi",
                                return.only.var.genes = FALSE, 
                                variable.features.n = 2000,
                                vst.flavor = "v2", verbose = FALSE)

snapshotDate(): 2022-10-31

loading from cache



Launching PCA.

In [15]:
hgsoc_merged.sct <- RunPCA(hgsoc_merged.sct, verbose = FALSE, assay.use = "SCT")

Integrating with Harmony. To do so, we need to specify the model variable as that corresponds to the batch we need to correct.

In [16]:
hgsoc_merged.sct@meta.data$model <- sapply(rownames(hgsoc_merged.sct@meta.data),
                                          function(x) strsplit(x = x, split = "_")[[1]][1])

In [17]:
hgsoc_merged.sct <- RunHarmony(hgsoc_merged.sct, assay.use = "SCT", group.by.vars = "model")

Harmony 1/10

Harmony 2/10

Harmony converged after 2 iterations

Warning message:
"Invalid name supplied, making object name syntactically valid. New object name is Seurat..ProjectDim.SCT.harmony; see ?make.names for more details on syntax validity"


UMAP.

In [18]:
hgsoc_merged.sct <- RunUMAP(hgsoc_merged.sct, 
                            reduction = "harmony", 
                            dims = 1:30, 
                            n.neighbors = 5, 
                            min.dist = 0.5, 
                            verbose = FALSE)

Warning message:
"The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session"


Saving the Seurat object for further use.

In [19]:
saveRDS(object = hgsoc_merged.sct, file = "HGSOC_CellHashing_noClustering.RDS")

In [20]:
sessionInfo()

R version 4.2.2 (2022-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.9 (Ootpa)

Matrix products: default
BLAS/LAPACK: /homedir01/adini22/.conda/envs/cellhashing_preprocessing/lib/libopenblasp-r0.3.21.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] harmony_0.1.1           Rcpp_1.0.10             viridis_0.6.2          
 [4] viridisLite_0.4.2       dplyr_1.1.2             tidyr_1.3.0            
 [7] ensembldb_2.22.0        AnnotationFilter_1.22.0 GenomicFeatures_1.50.3 
[10] AnnotationDbi_1